In [1]:
import numpy as np
import scipy as sp
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import metrics

In [3]:
import time
import os
import sys
import pickle
from minibatch import *

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
db_dir = "/home/sweet/2-coursework/725bmes/eeg_deeplearning/data_extraction/src"
ME_db_fname = "mesh_ME_db.pickle"
# MI_db_fname = "prelim_MI_db.pickle"

ME_db = {}
# MI_db = {}


t1 = time.time()
with open(db_dir + "/" + ME_db_fname, "rb") as f:
    ME_db = pickle.load(f)
print("Loaded ME database in %f s" % (time.time()-t1))

np.random.seed(0)
seq_perms = create_rand_seq_permutations(ME_db)
np.random.seed(0)
batch_gen = generate_batch(ME_db, seq_perms, 4)
output_seq, labels = next(batch_gen)
print(output_seq[0][0].shape)

Loaded ME database in 8.854981 s
(10, 11)


In [6]:
def build_conv2d(inputs, filters, kernel_size, strides=(1, 1), padding='same', activation=None, name=None, reuse=None):
    return tf.layers.conv2d(inputs=inputs, filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, activation=activation,
                kernel_initializer=tf.truncated_normal_initializer(stddev=0.1), bias_initializer=tf.constant_initializer(0.1), name=name, reuse=reuse)

In [134]:
def create_CNN_stack(nstack):
    input_layer = layers.Input(shape=(10,11,1))
    conv2d_L1 = layers.Conv2D(32, 3, activation="relu", padding="same", 
                              name="CNN_layer1"+"_"+str(nstack))(input_layer)
    conv2d_L2 = layers.Conv2D(64, 3, activation="relu", padding="same", 
                              name="CNN_layer2"+"_"+str(nstack))(conv2d_L1)
    conv2d_L3 = layers.Conv2D(128, 3, activation="relu", padding="same", 
                              name="CNN_layer3"+"_"+str(nstack))(conv2d_L2)
    flat = layers.Flatten()(conv2d_L3)
    dense = layers.Dense(1024, activation="relu", name="CNN_FC"+"_"+str(nstack))(flat)
    output_layer = layers.Dropout(dropout_rate, name="CNN_drop"+"_"+str(nstack))(dense)

    return input_layer, output_layer

def create_model_cascade():
    input_layers = []
    cnn_outputs = []

    for i in range(S):
        inp, out = create_CNN_stack(i)
        input_layers.append(inp)
        cnn_outputs.append(out)

    cnn_stack_model = models.Model(inputs=input_layers, outputs=cnn_outputs)

    concat_CNN_ouputs = layers.concatenate(cnn_stack_model.outputs)
    concat_CNN_ouputs = layers.Reshape((h_states_lstm, cnn_stack_model.outputs[0].shape[1]//h_states_lstm))(concat_CNN_ouputs)

    lstm1 = layers.LSTM(S, return_sequences=True, name="LSTM_1")(concat_CNN_ouputs)
    lstm2 = layers.LSTM(S, return_sequences=True, dropout=dropout_rate, name="LSTM_2")(lstm1)
    final_output = layers.Dense(K, activation="softmax", name="FC_softmax")(lstm2[:,:,-1])

    cascade_model = models.Model(inputs=input_layers, outputs=[final_output])
    
    return cascade_model

In [137]:
S = 2    # S = number of images
K = 7    # K = number of classes
h_states_lstm = 64
dropout_rate = 0.5

cascade_model = create_model_cascade()
cascade_model.summary()

Model: "model_30"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_74 (InputLayer)           [(None, 10, 11, 1)]  0                                            
__________________________________________________________________________________________________
input_75 (InputLayer)           [(None, 10, 11, 1)]  0                                            
__________________________________________________________________________________________________
CNN_layer1_0 (Conv2D)           (None, 10, 11, 32)   320         input_74[0][0]                   
__________________________________________________________________________________________________
CNN_layer1_1 (Conv2D)           (None, 10, 11, 32)   320         input_75[0][0]                   
___________________________________________________________________________________________

In [136]:
# inputs1 = layers.Input(shape=(3, 1))
# lstm1, state_h, state_c = layers.LSTM(1, return_sequences=True, return_state=True)(inputs1)
# model = models.Model(inputs=inputs1, outputs=[state_h])
# # define input data
# data = np.array([0.1, 0.2, 0.3]).reshape((1,3,1))
# # make and show prediction
# print(model.predict(data))